**Setup & Imports**

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain.vectorstores.utils import DistanceStrategy

from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os

In [ ]:
# install package
# !pip install -U langchain-ollama

**Load Text and Image Vector Stores**

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load text vectorstore
faiss_docs = FAISS.load_local(
    folder_path="../vectorstores/faiss_index_multimodal/faiss_docs",
    embeddings=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
    allow_dangerous_deserialization=True
)

# Load image-based OCR vectorstore
faiss_images = FAISS.load_local(
    folder_path="../vectorstores/faiss_index_multimodal/faiss_images",
    embeddings=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
    allow_dangerous_deserialization=True
)

print("Loaded FAISS indices for documents and OCR image data.")

Loaded FAISS indices for documents and OCR image data.


**Combine Text + Image FAISS into a Hybrid Retriever**

In [6]:
from langchain.retrievers import EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.vectorstores.base import VectorStoreRetriever

# Wrap each vectorstore as a retriever
text_retriever = VectorStoreRetriever(vectorstore=faiss_docs)
image_retriever = VectorStoreRetriever(vectorstore=faiss_images)

# Combine both with equal weights (can tune)
hybrid_retriever = EnsembleRetriever(
    retrievers=[text_retriever, image_retriever],
    weights=[0.5, 0.5]  # You can tune if one is more reliable
)

print("Hybrid retriever combining text and OCR vectorstores ready.")


Hybrid retriever combining text and OCR vectorstores ready.


**Build the RAG Chain with DeepSeek LLM**

In [ ]:
from langchain.chains import RetrievalQA
from langchain_ollama.llms import OllamaLLM

# Load DeepSeek model
llm = OllamaLLM(model="deepseek-r1:7b")

# Final RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # You can switch to refine or map_reduce for longer contexts
    retriever=hybrid_retriever,
    return_source_documents=True
)

print("Multimodal RAG chain ready for querying!")


Multimodal RAG chain ready for querying!


**Try a Query**

In [11]:
query = "How do I apply for a renovation permit from HDB? What documents do I need?"
response = rag_chain(query)

print("Answer:\n", response["result"])
print("\nSources:")
for doc in response["source_documents"]:
    print("-", doc.metadata.get("source", "unknown"))

C:\Users\Fleming Siow\AppData\Local\Temp\ipykernel_1440\3577143117.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain(query)


Answer:
 <think>
Okay, so I need to figure out how to apply for a renovation permit through HDB and what documents are needed. Let me try to go through the information provided step by step.

First, it says that applications can only be made electronically via APEX on HDB InfoWEB. So, I guess I have to use this online system instead of sending something in the mail or handing it over. The user mentioned that they should log into My HDBPage under "My Business > flat Renovation (For Contractors)" and then apply from there. That's probably where all the necessary information is entered electronically.

Now, the process seems to require two parts: Part III and IV of the form. I'm not exactly sure what these parts entail, but since it's a guide for using APEX, maybe they are specific sections or steps within the application. The user noted that this form is only applicable when APEX isn't available due to unforeseen issues. So, in normal circumstances, you should use Part III and IV of this